In [2]:
import pandas as pd
import os

In [5]:
pwd

'/home/nakyung/projects/BDAIFin/EDA&FEATURE'

In [6]:
df = pd.read_parquet("../DATA/dataset/TRAIN_stage1")

In [7]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

LABEL_COL = "fraud"  

y = df[LABEL_COL].astype(int)
X = df.drop(columns=[LABEL_COL])

# 1) 컬럼 타입 자동 추정

cat_cols = [c for c in X.columns if str(X[c].dtype) in ("object", "category")]
num_cols = [c for c in X.columns if c not in cat_cols]

print("num_cols:", len(num_cols), "cat_cols:", len(cat_cols))


# 2) Train/Valid split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) 간단 전처리 파이프

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)), 
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
    ],
    remainder="drop",
)


num_cols: 15 cat_cols: 0


In [8]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import shap
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.auto import tqdm

X_train_lgb = X_train.copy()
X_valid_lgb = X_valid.copy()

dtrain = lgb.Dataset(X_train_lgb, label=y_train, free_raw_data=False)
dvalid = lgb.Dataset(X_valid_lgb, label=y_valid, free_raw_data=False)

params = dict(
    objective="binary",
    metric=["auc", "average_precision"],
    learning_rate=0.05,
    num_leaves=64,
    min_data_in_leaf=200,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    verbosity=-1,
)

bst = lgb.train(
    params,
    dtrain,
    num_boost_round=5000,
    valid_sets=[dvalid],
    valid_names=["valid"],
    callbacks=[
        lgb.early_stopping(200, verbose=True),
        lgb.log_evaluation(0),
    ],
)

print("best_iter:", bst.best_iteration)
print("best_score:", bst.best_score)

pred_valid = bst.predict(X_valid_lgb, num_iteration=bst.best_iteration)
print("LGB AUC:", roc_auc_score(y_valid, pred_valid))
print("LGB PR-AUC:", average_precision_score(y_valid, pred_valid))


# SHAP with tqdm (batch version)

from tqdm.auto import tqdm

sv = X_valid_lgb

batch_size = 2000
all_contrib = []

print("\nComputing SHAP (LightGBM native)...")

for i in tqdm(range(0, len(sv), batch_size)):
    batch = sv.iloc[i:i+batch_size]
    contrib = bst.predict(batch, pred_contrib=True)
    all_contrib.append(contrib[:, :-1])  # 마지막 열은 bias

shap_values = np.vstack(all_contrib)

imp = pd.Series(
    np.abs(shap_values).mean(axis=0),
    index=sv.columns
).sort_values(ascending=False)

print("\nTop SHAP features:\n", imp.head(30))


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[533]	valid's auc: 0.896741	valid's average_precision: 0.0375346
best_iter: 533
best_score: defaultdict(<class 'collections.OrderedDict'>, {'valid': OrderedDict([('auc', np.float64(0.8967412943584248)), ('average_precision', np.float64(0.037534598363610956))])})
LGB AUC: 0.8967412943584248
LGB PR-AUC: 0.03753459836361088

Computing SHAP (LightGBM native)...


  0%|          | 0/534 [00:00<?, ?it/s]


Top SHAP features:
 tx_hour                   0.703894
hour_circular_distance    0.463239
hour_cos                  0.446013
mcc_highrisk_90           0.440688
log_abs_amount            0.440157
cos_shift                 0.229152
is_refund                 0.218054
is_highrisk_weekday       0.180360
sin_shift                 0.150146
tx_month                  0.145714
hour_sin                  0.138593
has_error                 0.005450
err_bad_cvv               0.002073
err_bad_card_number       0.001825
err_bad_expiration        0.001203
dtype: float64


In [9]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, average_precision_score

feature_names = list(X_train.columns)
n_features = len(feature_names)

scaler = StandardScaler()
Xtr = scaler.fit_transform(X_train.values.astype(np.float32))
Xva = scaler.transform(X_valid.values.astype(np.float32))

ytr = y_train.values.astype(np.float32)
yva = y_valid.values.astype(np.float32)

class TabDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_loader = DataLoader(TabDataset(Xtr, ytr), batch_size=4096, shuffle=True, num_workers=0)
valid_loader = DataLoader(TabDataset(Xva, yva), batch_size=8192, shuffle=False, num_workers=0)

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Attention layer (weights 반환)

class AttnEncoderLayer(nn.Module):
    def __init__(self, d_model=64, n_heads=4, dropout=0.1):
        super().__init__()
        self.mha = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.ln1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model),
            nn.Dropout(dropout),
        )
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x, return_attn=False):
        # x: [B, T, D]
        attn_out, attn_w = self.mha(x, x, x, need_weights=True, average_attn_weights=False)
        x = self.ln1(x + attn_out)
        x = self.ln2(x + self.ff(x))
        if return_attn:
            # attn_w: [B, heads, T, T]
            return x, attn_w
        return x


# 2) Tabular Transformer (CLS 토큰 사용)

class TabularAttentionModel(nn.Module):
    def __init__(self, n_features, d_model=64, n_heads=4, n_layers=2, dropout=0.1):
        super().__init__()
        self.n_features = n_features
        self.d_model = d_model

        # feature별 1->d 투영 (각 feature마다 별도의 linear)
        self.feat_proj = nn.ModuleList([nn.Linear(1, d_model) for _ in range(n_features)])

        # CLS token
        self.cls = nn.Parameter(torch.zeros(1, 1, d_model))
        nn.init.normal_(self.cls, std=0.02)

        self.layers = nn.ModuleList([AttnEncoderLayer(d_model, n_heads, dropout) for _ in range(n_layers)])

        self.head = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, 1),
        )

    def forward(self, X, return_attn=False):
        # X: [B, F]
        B, F = X.shape

        # feature tokens 만들기: [B, F, D]
        toks = []
        for j in range(F):
            xj = X[:, j:j+1]                 # [B, 1]
            toks.append(self.feat_proj[j](xj))  # [B, D]
        tok = torch.stack(toks, dim=1)       # [B, F, D]

        # CLS 붙이기: [B, 1+F, D]
        cls = self.cls.expand(B, -1, -1)
        x = torch.cat([cls, tok], dim=1)

        attn_all = []
        for layer in self.layers:
            if return_attn:
                x, attn = layer(x, return_attn=True)
                attn_all.append(attn)
            else:
                x = layer(x, return_attn=False)

        # CLS representation으로 예측
        cls_repr = x[:, 0, :]               # [B, D]
        logit = self.head(cls_repr).squeeze(1)

        if return_attn:
            return logit, attn_all  # list of [B, heads, T, T]
        return logit


# 3) 학습 루프

model = TabularAttentionModel(n_features=n_features, d_model=64, n_heads=4, n_layers=2, dropout=0.1).to(device)
opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
loss_fn = nn.BCEWithLogitsLoss()

def eval_model():
    model.eval()
    ps, ys = [], []
    with torch.no_grad():
        for xb, yb in valid_loader:
            xb = xb.to(device)
            logit = model(xb)
            prob = torch.sigmoid(logit).cpu().numpy()
            ps.append(prob)
            ys.append(yb.numpy())
    p = np.concatenate(ps)
    t = np.concatenate(ys)
    return roc_auc_score(t, p), average_precision_score(t, p)

EPOCHS = 5
for epoch in range(1, EPOCHS + 1):
    model.train()
    pbar = tqdm(train_loader, desc=f"train epoch {epoch}", leave=False)
    for xb, yb in pbar:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        logit = model(xb)
        loss = loss_fn(logit, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        pbar.set_postfix(loss=float(loss.detach().cpu()))

    auc, pr = eval_model()
    print(f"[epoch {epoch}] valid AUC={auc:.5f}  PR-AUC={pr:.5f}")


# 4) Attention 추출 → 컬럼 중요도
# - CLS(0번 토큰)에서 각 feature 토큰으로 가는 attention을 사용

def extract_feature_attention_importance(model, loader, n_batches=50):
    model.eval()
    # 누적: feature별 attention 합
    att_sum = np.zeros((n_features,), dtype=np.float64)
    cnt = 0

    with torch.no_grad():
        for b, (xb, yb) in enumerate(tqdm(loader, desc="extract attention", total=min(n_batches, len(loader)))):
            if b >= n_batches:
                break
            xb = xb.to(device)

            logit, attn_all = model(xb, return_attn=True)
            attn = attn_all[-1]  # [B, heads, T, T]

            # CLS -> feature 토큰 attention: query=0, key=1..F
            # shape: [B, heads, F]
            cls_to_feat = attn[:, :, 0, 1:]  

            # heads 평균, batch 평균 → [F]
            score = cls_to_feat.mean(dim=1).mean(dim=0).cpu().numpy()
            att_sum += score
            cnt += 1

    att_mean = att_sum / max(cnt, 1)
    imp = pd.Series(att_mean, index=feature_names).sort_values(ascending=False)
    return imp

att_imp = extract_feature_attention_importance(model, valid_loader, n_batches=50)
print("\nTop Attention features:\n", att_imp.head(30))


train epoch 1:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 1] valid AUC=0.87273  PR-AUC=0.02141


train epoch 2:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 2] valid AUC=0.88169  PR-AUC=0.02948


train epoch 3:   0%|          | 0/1042 [00:00<?, ?it/s]

...

[epoch 4] valid AUC=0.88676  PR-AUC=0.03287


train epoch 5:   0%|          | 0/1042 [00:00<?, ?it/s]

[epoch 5] valid AUC=0.88789  PR-AUC=0.03510


extract attention:   0%|          | 0/50 [00:00<?, ?it/s]


Top Attention features:
 log_abs_amount            0.156898
hour_sin                  0.146947
sin_shift                 0.120691
cos_shift                 0.085613
mcc_highrisk_90           0.081791
hour_cos                  0.077360
is_refund                 0.076557
tx_hour                   0.053810
tx_month                  0.040203
err_bad_expiration        0.034823
err_bad_card_number       0.034044
is_highrisk_weekday       0.026904
has_error                 0.021560
hour_circular_distance    0.015177
err_bad_cvv               0.008864
dtype: float64


In [10]:
compare = pd.DataFrame({
    "shap_mean_abs": imp,        
    "attn_cls2feat": att_imp
}).fillna(0.0)

compare["shap_rank"] = compare["shap_mean_abs"].rank(ascending=False, method="min")
compare["attn_rank"] = compare["attn_cls2feat"].rank(ascending=False, method="min")
compare["rank_gap"] = compare["attn_rank"] - compare["shap_rank"]

print(compare.sort_values("shap_mean_abs", ascending=False).head(40))


                        shap_mean_abs  attn_cls2feat  shap_rank  attn_rank  \
tx_hour                      0.703894       0.053810        1.0        8.0   
hour_circular_distance       0.463239       0.015177        2.0       14.0   
hour_cos                     0.446013       0.077360        3.0        6.0   
mcc_highrisk_90              0.440688       0.081791        4.0        5.0   
log_abs_amount               0.440157       0.156898        5.0        1.0   
cos_shift                    0.229152       0.085613        6.0        4.0   
is_refund                    0.218054       0.076557        7.0        7.0   
is_highrisk_weekday          0.180360       0.026904        8.0       12.0   
sin_shift                    0.150146       0.120691        9.0        3.0   
tx_month                     0.145714       0.040203       10.0        9.0   
hour_sin                     0.138593       0.146947       11.0        2.0   
has_error                    0.005450       0.021560       12.0 

### High

- tx_hour
- hour_cos
- hour_sin
- log_abs_amount
- mcc_highrisk_90
- cos_shift
- sin_shift


> 시간 & 금액이 강하게 작용\
> Error 계열은 약함

---

In [11]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm.auto import tqdm

params = dict(
    objective="binary",
    metric="auc",
    learning_rate=0.05,
    num_leaves=64,
    min_data_in_leaf=200,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    verbosity=-1,
)

def train_eval(X_tr, y_tr, X_va, y_va):
    dtrain = lgb.Dataset(X_tr, label=y_tr, free_raw_data=False)
    dvalid = lgb.Dataset(X_va, label=y_va, free_raw_data=False)

    bst = lgb.train(
        params,
        dtrain,
        num_boost_round=3000,
        valid_sets=[dvalid],
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
    )

    pred = bst.predict(X_va, num_iteration=bst.best_iteration)

    return {
        "auc": roc_auc_score(y_va, pred),
        "prauc": average_precision_score(y_va, pred),
        "best_iter": bst.best_iteration,
    }


In [12]:
base_result = train_eval(X_train, y_train, X_valid, y_valid)
print("BASE:", base_result)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[598]	valid_0's auc: 0.897014
BASE: {'auc': 0.897013976804593, 'prauc': 0.037417840743750264, 'best_iter': 598}


In [13]:
drop_one_results = []

for col in tqdm(X_train.columns):
    cols = [c for c in X_train.columns if c != col]

    res = train_eval(
        X_train[cols],
        y_train,
        X_valid[cols],
        y_valid,
    )

    drop_one_results.append({
        "dropped_feature": col,
        "auc_drop": base_result["auc"] - res["auc"],
        "prauc_drop": base_result["prauc"] - res["prauc"],
        "auc": res["auc"],
        "prauc": res["prauc"],
    })

drop_one_df = pd.DataFrame(drop_one_results)\
    .sort_values("auc_drop", ascending=False)

drop_one_df

  0%|          | 0/15 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[471]	valid_0's auc: 0.89614
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[400]	valid_0's auc: 0.896962
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[416]	valid_0's auc: 0.895368
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[142]	valid_0's auc: 0.894249
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[161]	valid_0's auc: 0.891889
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[567]	valid_0's auc: 0.896626
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[445]	valid_0's auc: 0.892308
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[419]	valid_0's

,dropped_feature,auc_drop,prauc_drop,auc,prauc
13,mcc_highrisk_90,0.044524,0.018001,0.852490,0.019417
7,log_abs_amount,0.016822,0.006314,0.880192,0.031104
12,hour_circular_distance,0.014662,0.008045,0.882352,0.029373
4,tx_month,0.005125,0.004357,0.891889,0.033061
14,is_highrisk_weekday,0.004835,0.003664,0.892179,0.033754
6,is_refund,0.004706,0.002103,0.892308,0.035315
3,err_bad_cvv,0.002765,0.006715,0.894249,0.030703
2,err_bad_expiration,0.001646,0.001634,0.895368,0.035784
10,sin_shift,0.001142,-0.001022,0.895872,0.038440
0,has_error,0.000873,0.000362,0.896140,0.037056


---